In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive/')

Direktori

In [ ]:
base_dir = '/content/drive/My Drive/AI/'
!ls "/content/drive/My Drive/AI/"

In [83]:
#dir All
bahan_dir = os.path.join(base_dir, 'bahan')
train_dir = os.path.join(base_dir, 'latih')
validation_dir = os.path.join(base_dir, 'validasi')

In [ ]:
#dir Bahan
rodadua_dir = os.path.join(bahan_dir, 'rodadua/')
rodaempat_dir = os.path.join(bahan_dir, 'rodaempat/')

print("Jumlah Data Train Tiap Kelas")
print('Jumlah gambar Kendaraan Roda Dua :', len(os.listdir(rodadua_dir)))
print('Jumlah gambar Kendaraan Roda Empat :', len(os.listdir(rodaempat_dir)))

In [85]:
#dir latih/train
train_rodadua = os.path.join(train_dir, 'rodadua/')
train_rodaempat = os.path.join(train_dir, 'rodaempat/')

#dir validasi
validation_rodadua = os.path.join(validation_dir, 'rodadua/')
validation_rodaempat = os.path.join(validation_dir, 'rodaempat/')

Prepare Data Set

In [86]:
import random
from shutil import copyfile

def train_val_split(source, train, val, train_ratio):
  total_size = len(os.listdir(source))
  train_size = int(train_ratio * total_size)
  val_size = total_size - train_size

  randomized = random.sample(os.listdir(source), total_size)
  train_files = randomized[0:train_size]
  val_files = randomized[train_size:total_size]

  for i in train_files:
    i_file = source + i
    destination = train + i
    copyfile(i_file, destination)

  for i in val_files:
    i_file = source + i
    destination = val + i
    copyfile(i_file, destination)

#ratio Train dan validasi
train_ratio = 0.9  


#training
source_00 = rodadua_dir
train_00 = train_rodadua
val_00 = validation_rodadua
train_val_split(source_00, train_00, val_00, train_ratio)

#validasi
source_01 = rodaempat_dir
train_01 = train_rodaempat
val_01 = validation_rodaempat
train_val_split(source_01, train_01, val_01, train_ratio)

In [ ]:
print('Jumlah All rodadua    :', len(os.listdir(rodadua_dir)))
print('Jumlah Train rodadua  :', len(os.listdir(train_rodadua)))
print('Jumlah Val rodadua    :', len(os.listdir(validation_rodadua)))

In [88]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [89]:
train_datagen = ImageDataGenerator(
                  rescale = 1./255,
                  rotation_range = 30,
                  horizontal_flip = True,
                  shear_range = 0.3,
                  fill_mode = 'nearest',
                  width_shift_range = 0.2,
                  height_shift_range = 0.2,
                  zoom_range = 0.1
)

val_datagen = ImageDataGenerator(
                  rescale = 1./255,
                  rotation_range = 30,
                  horizontal_flip = True,
                  shear_range = 0.3,
                  fill_mode = 'nearest',
                  width_shift_range = 0.2,
                  height_shift_range = 0.2,
                  zoom_range = 0.1
)

Target

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'categorical'
)

In [91]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') > 0.99):
      print('\nAkurasi mencapai 99%')
      self.model.stop_training = True

callbacks = myCallback()

**CNN**

In [92]:
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150,150, 3)),
          tf.keras.layers.MaxPooling2D(2, 2),
          tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
          tf.keras.layers.MaxPooling2D(2, 2),
          tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
          tf.keras.layers.MaxPooling2D(2, 2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(200, activation= 'relu'),
          tf.keras.layers.Dropout(0.3, seed=122),
          tf.keras.layers.Dense(500, activation= 'relu'),
          tf.keras.layers.Dropout(0.5, seed=122),
          tf.keras.layers.Dense(2, activation= 'sigmoid')
])

In [ ]:
model.summary()

In [94]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
            train_generator,
            steps_per_epoch = 9,
            epochs = 25,
            validation_data = val_generator,
            validation_steps = 1,
            verbose = 1,
            callbacks = [callbacks]
)

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.legend(loc = 'best')
plt.show()

plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and Validation accuracy')
plt.legend(loc = 'best')
plt.show()


Klasifikasi

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():

  path = fn
  img = image.load_img(path, target_size = (150, 150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 100)

  print(fn)
  
  class_list = os.listdir(train_dir)

  for j in range(42):
    if classes[0][j] == 1. :
      print('This image belongs to class', class_list[j-1])
      break